### LightGBM

Light GBM is a fast, distributed, high-performance gradient boosting framework based on decision tree algorithm, used for ranking, classification and many other machine learning tasks.

Since it is based on decision tree algorithms, it splits the tree leaf wise with the best fit whereas other boosting algorithms split the tree depth wise or level wise rather than leaf-wise. So when growing on the same leaf in Light GBM, the leaf-wise algorithm can reduce more loss than the level-wise algorithm and hence results in much better accuracy which can rarely be achieved by any of the existing boosting algorithms. Also, it is surprisingly very fast, hence the word ‘Light’.
<img src="LGBM.png">

<img src="">

In [1]:
import pandas as pd, numpy as np, time
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("/home/manishv/workspace/data/flights.csv")
data = data.sample(frac = 0.1, random_state=10)

In [3]:
data = data[["MONTH","DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT","AIR_TIME", "DEPARTURE_TIME","DISTANCE","ARRIVAL_DELAY"]]
data.dropna(inplace=True)

data["ARRIVAL_DELAY"] = (data["ARRIVAL_DELAY"]>10)*1

cols = ["AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT","ORIGIN_AIRPORT"]
for item in cols:
    data[item] = data[item].astype("category").cat.codes +1

data.head(5)

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,DESTINATION_AIRPORT,ORIGIN_AIRPORT,AIR_TIME,DEPARTURE_TIME,DISTANCE,ARRIVAL_DELAY
411984,1,28,3,14,102,717,608,102.0,713.0,634,0
3591965,8,11,2,3,152,748,690,134.0,111.0,1028,1
526451,2,4,3,4,1184,597,740,111.0,1734.0,931,0
1336011,3,27,5,14,170,770,609,173.0,1807.0,1436,0
3424502,8,1,6,14,4321,772,544,63.0,2151.0,481,1


In [4]:
train, test, y_train, y_test = train_test_split(data.drop(["ARRIVAL_DELAY"], axis=1), data["ARRIVAL_DELAY"],
                                                random_state=10, test_size=0.25)

In [5]:
import lightgbm as lgb

from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import cross_val_predict



def auc2(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict(train)),metrics.roc_auc_score(y_test,m.predict(test)))

lg = lgb.LGBMClassifier(silent=False)
param_dist = {"max_depth": [25,50, 75],
              "learning_rate" : [0.01,0.05,0.1],
              "num_leaves": [300,900,1200],
              "n_estimators": [200]
             }
grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="roc_auc", verbose=5)
grid_search.fit(train,y_train)
grid_search.best_estimator_



Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed: 21.2min finished


LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.05,
        max_bin=255, max_depth=50, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=200, nthread=-1,
        num_leaves=1200, objective='binary', reg_alpha=0, reg_lambda=0,
        seed=0, silent=False, subsample=1, subsample_for_bin=50000,
        subsample_freq=1)

In [8]:
d_train = lgb.Dataset(train, label=y_train)
params = {"max_depth": 50, "learning_rate" : 0.1, "num_leaves": 900}  #"n_estimators": 300}

# Without Categorical Features
model2 = lgb.train(params, d_train)
auc2(model2, train, test)

(0.908265702969921, 0.7781223108178189)

In [ ]:
d_train = lgb.Dataset(train, label=y_train)
#With Catgeorical Features
cate_features_name = ["MONTH","DAY","DAY_OF_WEEK","AIRLINE","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT"]
model2 = lgb.train(params, d_train, categorical_feature = cate_features_name)
auc2(model2, train, test)